In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\mages\\medical-chatbot-ai\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\mages\\medical-chatbot-ai'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#extract data from the PDF file

def load_pdf_file(data):
    loader=DirectoryLoader(
        data,glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#extracted_data

In [11]:
#split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 5860


In [14]:
#text_chunks

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings

In [25]:
#Download the Embedding from hugging face

def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings                               

In [26]:
embeddings=download_hugging_face_embedding()

c:\Users\mages\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mages\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [27]:
query_result= embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [29]:
#query_result

In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
Pinecone_API_Key=os.environ.get('PINECONE_API_KEY')

In [32]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=Pinecone_API_Key)

index_name="medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [34]:
import os
os.environ["PINECONE_API_KEY"]=Pinecone_API_Key

In [46]:
#embed each chunk and upsert the embedding into your pinecone index.

from langchain_pinecone import PineconeVectorStore


docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

ImportError: cannot import name 'PineconeVectorStore' from 'langchain_pinecone' (c:\Users\mages\anaconda3\envs\medibot\lib\site-packages\langchain_pinecone\__init__.py)